In [2]:
import pandas as pd
from transformers import pipeline, set_seed
from datasets import load_from_disk
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

##### Remainder for future:
<pre>Use Tensorboard to Visualize yor model</pre>
<pre>Change the model to T5</pre>
<pre>if you got system with NVDIA GPU increase Batch size and decrease<br> the Gradient_accumilation_steps(Hyper parameters)</pre>
<pre>Increase Number of epochs to 10 (currently set as 1)</pre>
<pre>its better if you push your model to Hub</pre>

In [3]:

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [8]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
dataset_samsum= load_from_disk("Datasets/samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

dataset_samsum["test"][1]["summary"]
     

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [21]:
def convert__to_features(batch):
    input_encodings = tokenizer(batch["dialogue"], max_length=1024,
                                truncation=True)
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(batch["summary"], max_length=128,
                                     truncation=True)
    
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

In [17]:
ids = tokenizer('Hello, how are you', max_length=1024,
                                truncation=True)
print(tokenizer.convert_ids_to_tokens( ids['input_ids']))
ids['input_ids']

['▁Hello', ',', '▁how', '▁are', '▁you', '</s>']


[8087, 108, 199, 127, 119, 1]

In [22]:
dataset_samsum_pt = dataset_samsum.map(convert__to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:01<00:00, 493.45 examples/s]


In [8]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [23]:
dataset_samsum_pt = dataset_samsum_pt.remove_columns(dataset_samsum["train"].column_names)
dataset_samsum_pt["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
seq2seq_data_collator

In [11]:

from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(output_dir='Datasets',num_train_epochs= 1, warmup_steps=500, per_device_train_batch_size= 1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps= 100, save_steps=500, gradient_accumulation_steps= 16, remove_unused_columns=False)

c:\Users\Rosha\code\textsummarizer\summarizerv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"], push_to_hub = True)

In [13]:
trainer.train()

 20%|█▉        | 10/51 [53:56<3:30:16, 307.71s/it]

{'loss': 3.1101, 'grad_norm': 31.987564086914062, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [1:43:57<2:30:14, 290.80s/it]

{'loss': 3.0467, 'grad_norm': 15.83983325958252, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


 59%|█████▉    | 30/51 [2:39:01<2:09:31, 370.09s/it]

{'loss': 3.1613, 'grad_norm': 10.380402565002441, 'learning_rate': 3e-06, 'epoch': 0.59}


 78%|███████▊  | 40/51 [3:32:39<1:06:35, 363.27s/it]

{'loss': 2.9903, 'grad_norm': 20.117599487304688, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


KeyboardInterrupt: 

In [15]:
model_pegasus.save_pretrained("model")
tokenizer.save_pretrained("model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('model\\tokenizer_config.json',
 'model\\special_tokens_map.json',
 'model\\spiece.model',
 'model\\added_tokens.json',
 'model\\tokenizer.json')